<a href="https://colab.research.google.com/github/Amelrich/Capstone-Fall-2020/blob/kassie-preprocessing/copy_of_Lawrence_main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [69]:
!pip install tslearn

     |████████████████████████████████| 778kB 3.1MB/s 


In [15]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pandas_datareader.data as web
from datetime import datetime
from random import sample
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score, KFold
from pandas import *
from sklearn.linear_model import LinearRegression
from scipy import stats
import seaborn as sns
import sys


In [13]:
import glob
import pandas as pd

# get data file names
path = '/content/drive/My Drive/capstone/data/'
filenames = glob.glob(path + "*.csv")

dfs = []
for filename in sorted(filenames):
    _=pd.read_csv(filename)
    _['Symbol']=filename.split('/')[-1].split('.')[0]
    _['Date'] = pd.to_datetime(_.Date)
    _ = _.sort_values(by=['Date']).reset_index(drop=True)
    dfs.append(_)

df_concat = pd.concat(dfs, ignore_index=True)
df_concat.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Symbol
0,1999-11-18,0.813058,0.813616,0.789621,0.800223,0.691504,364784000.0,AAPL
1,1999-11-19,0.799107,0.829241,0.786272,0.825335,0.713204,312513600.0,AAPL
2,1999-11-22,0.819196,0.819196,0.796875,0.809152,0.699220,202361600.0,AAPL
3,1999-11-23,0.819196,0.850446,0.790179,0.828683,0.716097,543312000.0,AAPL
4,1999-11-24,0.830357,0.848214,0.818638,0.845424,0.730564,215107200.0,AAPL


In [17]:
bwd_window=50; fwd_window = 20;
#smoothing = 3 
df = df_concat.copy()
df.loc[df.Volume==0,['Open','High','Low','Close','Volume']]=np.nan

df['Open']  = df['Open'] /df['Close']*df['Adj Close']
df['High']  = df['High'] /df['Close']*df['Adj Close']
df['Low']   = df['Low']  /df['Close']*df['Adj Close']
df['Close'] = df['Close']/df['Close']*df['Adj Close']

df = df[['Date','Symbol','Open','High','Low','Close','Volume']]
df.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  after removing the cwd from sys.path.


,Date,Symbol,Open,High,Low,Close,Volume
0,1999-11-18,AAPL,0.702595,0.703077,0.682342,0.691504,364784000.0
1,1999-11-19,AAPL,0.690540,0.716580,0.679449,0.713204,312513600.0
2,1999-11-22,AAPL,0.707900,0.707900,0.688611,0.699220,202361600.0
3,1999-11-23,AAPL,0.707900,0.734904,0.682824,0.716097,543312000.0
4,1999-11-24,AAPL,0.717544,0.732975,0.707417,0.730564,215107200.0


In [18]:
for fwd_window in [5,10,20]:
    df['fwd_max_close'] = df.iloc[::-1].groupby('Symbol', sort=False).rolling(fwd_window, on='Close', min_periods=fwd_window).Close.max().iloc[::-1].reset_index(name='field')['field']
    df['fwd_min_close'] = df.iloc[::-1].groupby('Symbol', sort=False).rolling(fwd_window, on='Close', min_periods=fwd_window).Close.min().iloc[::-1].reset_index(name='field')['field']
    df['CPP'+str(fwd_window)] = 100*df['fwd_max_close']/df['Close']-100
    df['CPL'+str(fwd_window)] = 100*df['fwd_min_close']/df['Close']-100
    
    df['fwd_max_high']  = df.iloc[::-1].groupby('Symbol', sort=False).rolling(fwd_window, on='High', min_periods=fwd_window).High.max().iloc[::-1].reset_index(name='field')['field']
    df['fwd_min_low']   = df.iloc[::-1].groupby('Symbol', sort=False).rolling(fwd_window, on='Low',  min_periods=fwd_window).Low.min().iloc[::-1].reset_index(name='field')['field']
    df['PP'+str(fwd_window)] = 100*df['fwd_max_high']/df['Open']-100
    df['PL'+str(fwd_window)] = 100*df['fwd_min_low']/df['Open']-100
    df = df.drop(columns=['fwd_max_close','fwd_min_close','fwd_max_high','fwd_min_low'])

df = df[(df.Date<'2015-01-01')&(df.Date>='1990-01-01')]
df = df[~df.isnull()]
df.head(4)

,Date,Symbol,Open,High,Low,Close,Volume,CPP5,CPL5,PP5,PL5,CPP10,CPL10,PP10,PL10,CPP20,CPL20,PP20,PL20
0,1999-11-18,AAPL,0.702595,0.703077,0.682342,0.691504,364784000.0,3877.487699,3655.625089,3971.235239,3583.185185,3901.544419,3610.183024,3971.235239,3491.105147,3901.544419,3535.337626,3971.235239,3389.817130
1,1999-11-19,AAPL,0.690540,0.716580,0.679449,0.713204,312513600.0,3722.776144,3541.356053,3895.088150,3647.486061,3779.793016,3497.296613,3982.082396,3553.798493,3779.793016,3424.728469,3982.082396,3433.344211
2,1999-11-22,AAPL,0.707900,0.707900,0.688611,0.699220,202361600.0,3799.232919,3653.836848,3797.117089,3581.697732,3857.390147,3569.243722,3881.977986,3464.196837,3857.390147,3435.745060,3881.977986,3346.696441
3,1999-11-23,AAPL,0.707900,0.734904,0.682824,0.716097,543312000.0,3707.331507,3513.737169,3797.116785,3542.529950,3764.118021,3482.762949,3881.977675,3464.196559,3764.118021,3344.666775,3881.977675,3301.000935


In [43]:
# df = pd.read_csv("/content/drive/My Drive/capstone/lawrence_data.csv")

In [ ]:
ind = list(df[(df['CPP20']>=10)&(df['CPP20']<=100)].index)
ind = sample(ind,len(ind))
X_data=[]; y_data=[];
s_num=2000;

for i in ind[:s_num]:
    x = df.loc[i-(bwd_window-1):i,:]['Close'].values
    s = df.loc[i-(bwd_window-1):i,:]['Symbol'].values
    y = df.loc[i-(bwd_window-1):i,:].index.values
    y = y[-1]-y[0]+1
    if np.sum(~np.isnan(x))==bwd_window and y==bwd_window and s[0]==s[-1]:
        X_data.append( (x-np.mean(x))/np.std(x) )
        y_data.append( df.loc[i,:]['CPP20'] )
        
X_data = np.array(X_data)
y_data = np.array(y_data)
print(X_data.shape, y_data.shape)

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
X_data, y_data = shuffle(X_data, y_data, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, random_state=42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [52]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.cluster import KMeans, SpectralClustering

clu = KMeans(n_clusters=n_clusters,random_state=0)

clu.fit( X_data ); 
clusters= clu.predict(X_data)

In [53]:
clusters= clu.predict(X_data)
labels1 = clu.labels_


In [63]:
dict_lawrence = {}

for i in range(0,10):
  dict_lawrence[i] = list()

for ind, arr in enumerate(X_data):
  dict_lawrence[clusters[ind]].append(arr)

In [64]:
clusters_n = [len(j) for j in dict_lawrence.values()]
clusters_n


[181, 158, 148, 153, 284, 237, 125, 187, 383, 118]

In [84]:
import tslearn
from tslearn.metrics import dtw, cdist_dtw

def DTW(cluster_array):
    n = len(cluster_array)
    total_distance = 0
    # Nested loop for all possible pairs
    for i in range(n):
        for j in range(n):
            total_distance = total_distance + dtw(cluster_array[i], cluster_array[j])

    return total_distance
total = 0
for i in range(0,10):
  d = DTW(dict_lawrence[i])
  total = total + d
  print("Cluster {} has distance {}".format(i, d))

Cluster 0 has distance 127417.1792664453
Cluster 1 has distance 89695.237079943
Cluster 2 has distance 85449.85917513732
Cluster 3 has distance 88587.83110967228
Cluster 4 has distance 264302.75707859517
Cluster 5 has distance 180487.2087382034
Cluster 6 has distance 67269.33126323386
Cluster 7 has distance 145403.95804640866
Cluster 8 has distance 384455.9462651699
Cluster 9 has distance 57771.28409518629


In [85]:
total

1490840.592117995